In [5]:
import numpy as np
import math
from typing import List
import os
import argparse
import glob
import shutil
from torchvision import transforms
import torchvision.datasets as datasets
import torch.utils.data as loader
from torchvision import models
import torch
import torch.nn as nn
import time
import torch.optim as optim
from PIL import Image
import requests
import matplotlib.pyplot as plt

In [18]:
image_transforms = {
    # 'train': transforms.Compose([
    #           transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
    #           transforms.RandomRotation(degrees=15),
    #           transforms.RandomHorizontalFlip(),
    #           transforms.CenterCrop(size=224),
    #           transforms.ToTensor(),
    #           transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    # ]),
    # 'validation': transforms.Compose([
    #          transforms.Resize(size=256),
    #          transforms.CenterCrop(size=224),
    #          transforms.ToTensor(),
    #          transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])              
    # ]),
    'test': transforms.Compose([
             transforms.Resize(size=256),
             transforms.CenterCrop(size=224),
             transforms.ToTensor(),
             transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])              
    ])
}

In [28]:
# Create iterator for the data loader using DataLoader module

batch_size = 10

data = {
    # 'train': datasets.ImageFolder(root='./101_ObjectCategories_sets/train', transform=image_transforms['train']),
    # 'validation': datasets.ImageFolder(root='./101_ObjectCategories_sets/validation',transform=image_transforms['validation']),
    'test': datasets.ImageFolder(root='./101_ObjectCategories_sets/test', transform=image_transforms['test'])
}

In [22]:
model = models.resnet50(pretrained=False)
fc_inputs = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(fc_inputs, 2048),
    nn.ReLU(inplace=True),
    nn.Linear(2048, 10),
    nn.Dropout(0.4),
    nn.LogSoftmax(dim=1))

if(torch.cuda.is_available()):
    model = model.to("cuda")
model.load_state_dict(torch.load('model_0.pth'))

<All keys matched successfully>

In [23]:
index_to_class = {v: k for k, v in data['test'].class_to_idx.items()}
print (index_to_class)

def makePrediction(model, url):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = image_transforms['test']

    # test_image = Image.open(requests.get(url, stream=True).raw)
    test_image = Image.open(url)

    plt.imshow(test_image)
    
    test_image_tensor = transform(test_image)
    test_image_tensor = test_image_tensor.view(1, 3, 224, 224).to(device)
    with torch.no_grad():
        model.eval()
        out = model(test_image_tensor)
        ps = torch.exp(out)
        
        topk, topclass = ps.topk(3, dim=1)
        for i in range(3):
            print(f"Prediction {i+1} : {index_to_class[topclass.cpu().numpy()[0][i]]}, Score: {topk.cpu().numpy()[0][i] * 100}%")

{0: 'Leopards', 1: 'airplanes', 2: 'butterfly', 3: 'camera', 4: 'elephant', 5: 'lamp', 6: 'rhino', 7: 'umbrella', 8: 'watch'}


In [3]:
makePrediction(model, './image_0012.jpg')


NameError: name 'makePrediction' is not defined

In [1]:
makePrediction(model, './myWatch.jpg')

NameError: name 'makePrediction' is not defined